# (one-step) Biot equation

In this tutorial we present how to solve the (static) Biot equation with [PyGeoN](https://github.com/compgeo-mox/pygeon).  The unknown is the displacement $u$, the flux $q$ and the pressure $p$.

Let $\Omega$ with boundary $\partial \Omega$ and outward unit normal ${\nu}$. Given 
$\lambda$ Lamé constant and $\mu$ the Kirchhoff modulus, $K$ the permeability of the porous medium and $s_0$ its storativity, and $\alpha$ the Biot-Willis coefficient fot the coupling between the two models.
We want to solve the following problem: find $({u}, {q}, p)$ such that
$$
    \begin{aligned}
         & \nabla \cdot [2 \mu {\epsilon}({u}) + \lambda \nabla \cdot u I
         - \alpha p {I}] = -{b}   \\
         & \mu{q} + {K}\nabla p = {0}                        \\
         & \partial_t (s_0 p + \alpha \nabla \cdot {u}) +
        \nabla \cdot {q} = \psi
    \end{aligned}
    \quad \text{in } \Omega.
$$
with $\epsilon$ the symmetric gradient and $b$ a body force. The stress tensor, which can be post-processed from $u$, is given by
$$
    \sigma = 2 \mu \epsilon(u) + \lambda \nabla \cdot u I - \alpha p I
$$

In particular, we consider only the spatial structure of the previous problem by doing only one time step. We call the latter $\Delta t$ and write the discrete problem as
$$
    \begin{bmatrix}
        K & 0 & -\alpha D^\top \\
        0 & \Delta t M_q & - \Delta t B^\top \\
        \alpha D & \Delta t B & s_0 M_p
    \end{bmatrix}
    \begin{bmatrix}
        u \\ q \\ p
    \end{bmatrix}
    =
    \begin{bmatrix}
        b \\ 0 \\ \psi
    \end{bmatrix}
$$
where $K$ is the stiffness matrix associated with the elatic problem, $D$ is the coupling between the two physics, $M_q$ is the mass matrix associated to the flux variable, $B$ is the divergence matrix for the flow problem, and $M_p$ is the mass matrix associated with the pressure. All the aforementioned matrices are properly scaled by their physical parameters if not explicitly written. The second row has been multiply by $\Delta t$ to preserve the skew-symmetry of the problem.

## Exercise 1: fixed-strain iterative solver

To solve the discrete problem we employ the fixed-strain iterative solver which reads:
given $u^i$ compute $(u^{i+1}, q^{i+1}, p^{i+1})$ by solving until convergence the following steps:

$$
 \begin{bmatrix}
         \Delta t M_q & - \Delta t B^\top \\
         \Delta t B & s_0 M_p
    \end{bmatrix}
    \begin{bmatrix}
         q^{i+1} \\ p^{i+1}
    \end{bmatrix}
    =
    \begin{bmatrix}
       0 \\ \psi- D\alpha u^{i}
    \end{bmatrix}
$$

followed by
$$
K u^{i+1} = b + \alpha D^\top p^{i+1}
$$

Note that i) we are performing only one time step starting from $u=0$, $p=0$, therefore the right hand side is simplified and ii) the two system are equivalent to the following block triangular one:

$$
    \begin{bmatrix}
        K & 0 & -\alpha D^\top \\
        0 & \Delta t M_q & - \Delta t B^\top \\
        0 & \Delta t B & s_0 M_p
    \end{bmatrix}
    \begin{bmatrix}
        u^{i+1} \\ q^{i+1} \\ p^{i+1}
    \end{bmatrix}
    =
    -
    \begin{bmatrix}
        0 & 0 & 0 \\
        0 & 0 & 0 \\
        \alpha D& 0 & 0
    \end{bmatrix}
    \begin{bmatrix}
        u^{i} \\ q^{i} \\ p^{i}
    \end{bmatrix}
    +
    \begin{bmatrix}
        b \\ 0 \\ \psi
    \end{bmatrix}
$$

A footing problem is when a force is impose on the top compressing the body and the bottom is fixed. In this case we assume that the force is applied on the central portion of the top boundary.

For this test case we set $\Omega = [0, 3] \times [0, 1]$, $b = 0$, and the following boundary conditions:
$$ 
\begin{aligned}
    &u = 0 &&\text{ and }&& \nu \cdot q = 0 &&\text{ on } \partial \Omega  \setminus \partial_{top} \Omega
\\
&\nu \cdot \sigma = 0 &&\text{ and }&& p = 0 &&\text{ on } \partial_{top} \Omega \setminus \partial_s \Omega
\\
&\nu \cdot \sigma = [0, -1]^\top &&\text{ and }&& p = 0 &&\text{ on }\partial_s \Omega 
\end{aligned}
$$
where $\partial_{s} \Omega = [1, 2] \times \{1\}$.

We present *step-by-step* how to create the grid, declare the problem data, and finally solve the problem.

First we import some of the standard modules, like `numpy` and `scipy.sparse`. Since PyGeoN is based on [PorePy](https://github.com/pmgbergen/porepy) we import both modules.

In [1]:
import numpy as np
import scipy.sparse as sps

import porepy as pp
import pygeon as pg


# useful for the stopping criteria in the iterative solvers
def compute_err(x, x_i, mass):
    diff = np.sqrt((x_i - x) @ mass @ (x_i - x))
    norm = np.sqrt(x_i @ mass @ x_i)
    return diff / norm if norm else diff

We create now the grid, since we use a vector Lagrangian of order 1 for ${u}$ and Raviart-Thomas for the flux $q$, we are restricted to simplices. In this example we consider a 2-dimensional structured grid, but the presented code will work also in 3d.

In [2]:
mesh_size = 0.05
delta_t = 0.01
dim = 2

nx = np.array([30, 10])
sd = pp.StructuredTriangleGrid(nx, [1] * dim)
sd.nodes[0, :] *= 3

pg.convert_from_pp(sd)
sd.compute_geometry()

# data for the iterative solvers
tol = 1e-4
max_iter = 1e2

With the following code we set the data and the boundary conditions. Since we need to identify each side of $\partial \Omega$ we need few steps.

In [3]:
key = "biot"

# the physical parameters of the problem, assumed constant
lambda_ = 1
mu = 0.5
alpha = 1
s0 = 0.5  # 1 0.1

data = {}
inv_perm = pp.SecondOrderTensor(np.ones(sd.num_cells))
param = {"second_order_tensor": inv_perm, "lambda": lambda_, "mu": mu}
pp.initialize_data(sd, data, key, param)

# selection of the boundary conditions
bd_q = sd.tags["domain_boundary_faces"]
bd_q[np.isclose(sd.face_centers[1, :], 1)] = False

bd_u = sd.tags["domain_boundary_nodes"]
bd_u[np.isclose(sd.nodes[1, :], 1)] = False
bd_u = np.hstack([bd_u] * dim)

top_s = np.logical_and.reduce(
    (
        np.isclose(sd.face_centers[1, :], 1),
        sd.face_centers[0, :] > 1,
        sd.face_centers[0, :] < 2,
    )
)

fun = lambda _: np.array([0, -1])

Once the data are assigned to the grid, we construct the matrices. Once the latter is created, we also construct the right-hand side containing the boundary conditions.

In [4]:
# definition of the discretizations
vec_p1 = pg.VecLagrange1(key)
p0 = pg.PwConstants(key)
rt0 = pg.RT0(key)

# dofs for the different variables
dof_u = sd.num_nodes * dim
dof_q = sd.num_faces
dof_p = sd.num_cells
dofs = np.cumsum([dof_u, dof_q, dof_p])

# construction of the block matrices
mass_u = vec_p1.assemble_mass_matrix(sd)
mass_q = rt0.assemble_mass_matrix(sd)
mass_p = p0.assemble_mass_matrix(sd)
div_q = mass_p @ rt0.assemble_diff_matrix(sd)

sym_sym = vec_p1.assemble_symgrad_symgrad_matrix(sd)
div_div = vec_p1.assemble_div_div_matrix(sd)
div_u = mass_p @ vec_p1.assemble_div_matrix(sd)

stiff_u = 2 * mu * sym_sym + lambda_ * div_div

We need to solve the linear system, PyGeoN provides a framework for that. The actual imposition of essential boundary conditions (displacement boundary conditions) might change the symmetry of the global system, the class `pg.LinearSystem` preserves this structure by internally eliminating these degrees of freedom.

In [5]:
# construction of the matrices for the sub_problems and the RHS (the fixed part)
# fmt: off
sp_fluid = sps.block_array([[delta_t * mass_q, -delta_t * div_q.T],
                            [ delta_t * div_q,        s0 * mass_p]], format="csc")

# fmt: on

sp_mech = stiff_u

bd_mech = vec_p1.assemble_nat_bc(sd, fun, top_s)
bd_fluid = np.hstack((bd_q, np.zeros(dof_p, dtype=bool)))

step = 0
err = tol + 1

# initialization of the solution
u_i = np.zeros(dof_u)
q_i = np.zeros(dof_q)
p_i = np.zeros(dof_p)

while err > tol and step < max_iter:
    # for a given u solve the flow problem
    rhs_fluid = np.zeros(dof_q + dof_p)
    rhs_fluid[-dof_p:] = -alpha * div_u @ u_i

    ls1 = pg.LinearSystem(sp_fluid, rhs_fluid)
    ls1.flag_ess_bc(bd_fluid, np.zeros_like(rhs_fluid))
    x = ls1.solve()
    # split of the solution from the vector x
    q, p = np.split(x, [dof_q])

    # for a given p solve the mech problem
    rhs_mech = bd_mech + alpha * div_u.T @ p

    ls2 = pg.LinearSystem(sp_mech, rhs_mech)
    ls2.flag_ess_bc(bd_u, np.zeros_like(rhs_mech))
    u = ls2.solve()

    # compute the stopping criteria
    step += 1
    err_u = compute_err(u_i, u, mass_u)
    err_q = compute_err(q_i, q, mass_q)
    err_p = compute_err(p_i, p, mass_p)

    u_i = u.copy()
    q_i = q.copy()
    p_i = p.copy()

    err = err_u + err_q + err_p

    print(step, err, err_u, err_q, err_p)

1 1.0 1.0 0.0 0.0
2 2.838586363988022 0.8385863639880224 1.0 1.0
3 3.204215398057654 0.38723335693223215 1.1890117503782147 1.6279702907472071
4 1.31425344875812 0.3069697194582774 0.47799145182942177 0.5292922774704206
5 1.1178989670144843 0.17542194613891338 0.42804105184150965 0.5144359690340613
6 0.618442556435814 0.12536262957920608 0.234766273473845 0.25831365338276296
7 0.4583753458210344 0.07792842606796346 0.17831554784173156 0.2021313719113394
8 0.281410010455167 0.05328983277995742 0.10904327008743878 0.11907690758777084
9 0.19588078763047087 0.03431469736457472 0.07678107190205345 0.0847850183638427
10 0.12551834123283595 0.02302791573109616 0.04912858419620431 0.05336184130553549
11 0.08495200656437978 0.015055844121891917 0.03343497210655614 0.03646119033593172
12 0.05544390779540191 0.01002272567447358 0.021813346109788016 0.023607836011140318
13 0.03706089953387678 0.0065970014159480025 0.014620545508889814 0.01584335260903896
14 0.024380653893476347 0.00437665193152283

We post-process now the stress tensor, one `sd.dim` x `sd.dim` tensor per cell.

In [6]:
cell_sigma = vec_p1.compute_stress(sd, u, data)

# split the tensor in each component for the exporting
cell_sigma_xx = cell_sigma[:, 0, 0]
cell_sigma_xy = cell_sigma[:, 0, 1]
cell_sigma_yy = cell_sigma[:, 1, 1]

Since the computed $u$ is a vector per peak of the grid, for visualization purposes we project the displacement in each cell center as vector. Similarly for the flow field $q$. We finally export the solution to be visualized by [ParaView](https://www.paraview.org/).

In [7]:
# compute the cell flow, one vector per cell
proj_q = rt0.eval_at_cell_centers(sd)
cell_q = (proj_q @ q).reshape((3, -1))
cell_p = p0.eval_at_cell_centers(sd) @ p
u = np.hstack((u, np.zeros(sd.num_nodes))).reshape((3, -1))

# export the final solution
save = pp.Exporter(sd, "sol", folder_name="ex1")
save.write_vtu(
    [
        ("cell_q", cell_q),
        ("cell_p", cell_p),
        ("cell_sigma_xx", cell_sigma_xx),
        ("cell_sigma_xy", cell_sigma_xy),
        ("cell_sigma_yy", cell_sigma_yy),
    ],
    data_pt=[("u", u)],
)

In [8]:
# compute the spectral radius of the iteration matrix
# preconditioner P
# fmt: off
P = sps.block_array(
    [
        [stiff_u,             None,   -alpha * div_u.T],
        [   None, delta_t * mass_q, -delta_t * div_q.T],
        [   None,  delta_t * div_q,        s0 * mass_p],
    ]
)

# P-A
M = sps.block_array(
    [
        [   0 * stiff_u,       None,       None],
        [          None, 0 * mass_q,       None],
        [-alpha * div_u,       None, 0 * mass_p],
    ]
)
# fmt: on
# compute the 6 largest magnitude eigs for the generalized problem M x = lambda P x
ll = sps.linalg.eigs(M, 6, P, None, "LM")
print(ll)

(array([-0.76743539+0.j, -0.70954175+0.j, -0.62986239+0.j, -0.54369724+0.j,
       -0.48177616+0.j, -0.46176293+0.j]), array([[-1.72416914e-01+0.j, -7.23215550e-02+0.j, -5.34053235e-02+0.j,
         1.62195960e-02+0.j,  3.91724278e-01+0.j,  5.05963275e-02+0.j],
       [-1.71901538e-01+0.j, -7.35766980e-02+0.j, -5.55925882e-02+0.j,
         1.92479746e-02+0.j,  3.90549076e-01+0.j,  5.39666814e-02+0.j],
       [-1.69358389e-01+0.j, -7.92325750e-02+0.j, -6.43619502e-02+0.j,
         2.99068753e-02+0.j,  3.86300628e-01+0.j,  6.41774006e-02+0.j],
       ...,
       [ 2.23581861e-04+0.j,  4.48574655e-04+0.j, -6.41957597e-04+0.j,
        -7.53407701e-04+0.j,  3.74567736e-04+0.j,  7.12064936e-04+0.j],
       [ 1.00035766e-04+0.j,  2.06802360e-04+0.j, -3.10432942e-04+0.j,
        -3.89894681e-04+0.j,  1.65509971e-04+0.j,  4.11097531e-04+0.j],
       [ 1.01300334e-04+0.j,  2.08254690e-04+0.j, -3.09630590e-04+0.j,
        -3.83087091e-04+0.j,  1.75646994e-04+0.j,  3.92571417e-04+0.j]]))


In [11]:
# Consistency check
assert np.isclose(np.linalg.norm(cell_q), 11.030897445172489)
assert np.isclose(np.linalg.norm(cell_p), 3.6574422852272135)
assert np.isclose(np.linalg.norm(cell_sigma), 6.031072715738311)
assert np.isclose(np.linalg.norm(u), 1.2629371925588575)